In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import seaborn as sns
import warnings
import re
!pip install googlemaps
import googlemaps
gmaps = googlemaps.Client(key='REMOVED FOR SHARING')

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40716 sha256=f289ab15ffa149923fcfd3230d915437b9883f64c860d68396e3ccf84e845ed4
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
# setting configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
# linking google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 01. Importing & Cleaning Data

## A. Iowa Liquor Sales

### Importing Data

In [ ]:
# importing liquor data
ls_df = pd.read_csv('/content/drive/MyDrive/Iowa Liquor /Raw Data/iowa_liquor_sales.csv')
ls_df.shape

(2649227, 29)

In [ ]:
ls_df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Iowa ZIP Code Tabulation Areas,Iowa Watershed Sub-Basins (HUC 08),Iowa Watersheds (HUC 10),County Boundaries of Iowa,US Counties
0,INV-48886500007,07/04/2022,2502,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021.0,NaN,77.0,POLK,1032100,IMPORTED VODKAS,260,DIAGEO AMERICAS,34458,KETEL ONE,6,1750,20.99,31.49,6,188.94,10.5,2.77,NaN,NaN,NaN,NaN,NaN
1,INV-48886000004,07/04/2022,4129,CYCLONE LIQUORS,626 LINCOLN WAY,AMES,50010.0,POINT (-93.618289 42.022865),85.0,STORY,1012100,CANADIAN WHISKIES,259,HEAVEN HILL BRANDS,11788,BLACK VELVET,6,1750,10.45,15.68,6,94.08,10.5,2.77,609.0,11.0,124.0,53.0,299.0
2,INV-48886500013,07/04/2022,2502,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021.0,NaN,77.0,POLK,1031200,AMERICAN FLAVORED VODKA,260,DIAGEO AMERICAS,77994,"SMIRNOFF RED, WHITE & BERRY",6,1750,14.75,22.13,6,132.78,10.5,2.77,NaN,NaN,NaN,NaN,NaN
3,INV-48886600001,07/04/2022,5444,MARSHALL BEER WINE SPIRITS,11 N 3RD AVE,MARSHALLTOWN,50158.0,POINT (-92.908013 42.050162),64.0,MARSHALL,1011200,STRAIGHT BOURBON WHISKIES,368,PARK STREET IMPORTS,28137,WIDOW JANE 10YR BOURBON WHISKEY,6,750,40.00,60.00,6,360.00,4.5,1.18,89.0,21.0,277.0,54.0,1831.0
4,INV-48886300007,07/04/2022,4025,KARAM KAUR KHASRIYA LLC,702 13TH ST,BELLE PLAINE,52208.0,POINT (-92.277759 41.897052),6.0,BENTON,1031100,AMERICAN VODKAS,301,FIFTH GENERATION INC,38174,TITOS HANDMADE VODKA,12,375,5.42,8.13,12,97.56,4.5,1.18,261.0,21.0,281.0,46.0,1367.0


### Cleaning Data

In [ ]:
# create a dictionary to map old column names to new column names
new_column_names = {col: col.lower().replace(' ', '_') for col in ls_df.columns}

# rename the columns using the dictionary
ls_df.rename(columns=new_column_names, inplace=True)

In [ ]:
# checking for missing values
ls_df.isnull().sum()

invoice/item_number                         0
date                                        0
store_number                                0
store_name                                  0
address                                  1995
city                                     1995
zip_code                                 2017
store_location                         110366
county_number                         2550581
county                                   1995
category                                    0
category_name                               0
vendor_number                               0
vendor_name                                 0
item_number                                 0
item_description                            0
pack                                        0
bottle_volume_(ml)                          0
state_bottle_cost                           0
state_bottle_retail                         0
bottles_sold                                0
sale_(dollars)                    

In [ ]:
# filling in missing values for relevant columns by looking up the relevant values from the corresponding store number.

missing_columns = ['store_name', 'address', 'city', 'zip_code', 'county_number', 'county']

# fill missing values using the first valid observation within each group
ls_df[missing_columns] = ls_df.groupby('store_number')[missing_columns].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

ls_df.isnull().sum()

invoice/item_number                        0
date                                       0
store_number                               0
store_name                                 0
address                                    0
city                                       0
zip_code                                   0
store_location                        110366
county_number                         214138
county                                     0
category                                   0
category_name                              0
vendor_number                              0
vendor_name                                0
item_number                                0
item_description                           0
pack                                       0
bottle_volume_(ml)                         0
state_bottle_cost                          0
state_bottle_retail                        0
bottles_sold                               0
sale_(dollars)                             0
volume_sol

In [ ]:
# converting the zip code to int rather than float
ls_df['zip_code'] = ls_df['zip_code'].fillna(0).astype(int)

In [ ]:
# creating list of addresses to lookup coordinates for
ls_df['unique_store'] =  ls_df['address'] + ', ' + ls_df['city'] + ', ' + ls_df['zip_code'].astype(str)
unique_stores = ls_df['unique_store'].unique()
unique_stores.shape

(2090,)

In [ ]:
# looking up coordinates for stores

store_coordinates = {}

for store in unique_stores:
    geocode_result = gmaps.geocode(store)
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        store_coordinates[store] = (location['lat'], location['lng'])
    else:
        print(f"Failed to geocode: {store}")
        store_coordinates[store] = (np.nan, np.nan)

Failed to geocode: 603  E HWY 24, LAWLER, 52154
Failed to geocode: 1350 11TH ST NW, CLINTON, 52733
Failed to geocode: 101 E. HWY 7, ALTA, 51002
Failed to geocode: 1706 JEFFREY'S DR. PO BOX 466, OSCEOLA, 50213
Failed to geocode: 611 MAIN ST, DUMONT, 50602
Failed to geocode: 702 US HWY 65, COLO, 50056
Failed to geocode: 303 E HWY 7, ALTA, 51002
Failed to geocode: 109 E HWY 2, MILTON, 52570
Failed to geocode: 110, E JEFFERSON ST, DELAWARE, 52036


In [ ]:
# add missing coordinates
missing_coordinates = {
    "603  E HWY 24, LAWLER, 52154": (43.06820447630154, -92.14756287116384),
    '1350 11TH ST NW, CLINTON, 52733': (41.86109115225438, -90.21259650000007),
    '101 E. HWY 7, ALTA, 51002': (42.67660710691385, -95.303798),
    '611 MAIN ST, DUMONT, 50602': (42.751048123342535, -92.97295282327302),
    "1706 JEFFREY'S DR. PO BOX 466, OSCEOLA, 50213": (41.027570746894135, -93.78928752895436),
    '702 US HWY 65, COLO, 50056': (42.0150435672536, -93.31002495581926),
    "303 E HWY 7, ALTA, 51002": (42.67574568247349, -95.30199993545983),
    "109 E HWY 2, MILTON, 52570": (40.667990204700835, -92.16089222896322),
    "110, E JEFFERSON ST, DELAWARE, 52036": (42.47368751257071, -91.34483144232895),
}

store_coordinates.update(missing_coordinates)

In [ ]:
# map coordinates back to ls_df
ls_df['store_latitude'] = ls_df['unique_store'].map(lambda x: store_coordinates.get(x, (np.nan, np.nan))[0])
ls_df['store_longitude'] = ls_df['unique_store'].map(lambda x: store_coordinates.get(x, (np.nan, np.nan))[1])

In [ ]:
ls_df.isnull().sum()

invoice/item_number                        0
date                                       0
store_number                               0
store_name                                 0
address                                    0
city                                       0
zip_code                                   0
store_location                        110366
county_number                         214138
county                                     0
category                                   0
category_name                              0
vendor_number                              0
vendor_name                                0
item_number                                0
item_description                           0
pack                                       0
bottle_volume_(ml)                         0
state_bottle_cost                          0
state_bottle_retail                        0
bottles_sold                               0
sale_(dollars)                             0
volume_sol

In [ ]:
# drop columns not needed for analysis
ls_df = ls_df.drop(columns=['invoice/item_number', 'store_number', 'county_number', 'category',
                            'vendor_number', 'vendor_number', 'item_number', 'volume_sold_(gallons)',
                            'iowa_zip_code_tabulation_areas', 'iowa_watershed_sub-basins_(huc_08)',
                            'iowa_watersheds_(huc_10)', 'county_boundaries_of_iowa', 'us_counties',
                            'store_location' ])
ls_df.head()

,date,store_name,address,city,zip_code,county,category_name,vendor_name,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),unique_store,store_latitude,store_longitude
0,07/04/2022,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021,POLK,IMPORTED VODKAS,DIAGEO AMERICAS,KETEL ONE,6,1750,20.99,31.49,6,188.94,10.5,"410 NORTH ANKENY BLVD, ANKENY, 50021",41.734504,-93.602470
1,07/04/2022,CYCLONE LIQUORS,626 LINCOLN WAY,AMES,50010,STORY,CANADIAN WHISKIES,HEAVEN HILL BRANDS,BLACK VELVET,6,1750,10.45,15.68,6,94.08,10.5,"626 LINCOLN WAY, AMES, 50010",42.021077,-93.618292
2,07/04/2022,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021,POLK,AMERICAN FLAVORED VODKA,DIAGEO AMERICAS,"SMIRNOFF RED, WHITE & BERRY",6,1750,14.75,22.13,6,132.78,10.5,"410 NORTH ANKENY BLVD, ANKENY, 50021",41.734504,-93.602470
3,07/04/2022,MARSHALL BEER WINE SPIRITS,11 N 3RD AVE,MARSHALLTOWN,50158,MARSHALL,STRAIGHT BOURBON WHISKIES,PARK STREET IMPORTS,WIDOW JANE 10YR BOURBON WHISKEY,6,750,40.00,60.00,6,360.00,4.5,"11 N 3RD AVE, MARSHALLTOWN, 50158",42.050160,-92.907726
4,07/04/2022,KARAM KAUR KHASRIYA LLC,702 13TH ST,BELLE PLAINE,52208,BENTON,AMERICAN VODKAS,FIFTH GENERATION INC,TITOS HANDMADE VODKA,12,375,5.42,8.13,12,97.56,4.5,"702 13TH ST, BELLE PLAINE, 52208",41.897253,-92.277697


In [ ]:
# store coordinates should fall within "min_lat": 40.3622, "max_lat": 43.5008, "min_lng": -96.6357, "max_lng": -90.1538" otherwise they are incorrect.

max_value_lat = ls_df['store_latitude'].max()
min_value_lat = ls_df['store_latitude'].min()
max_value_lng = ls_df['store_longitude'].max()
min_value_lng = ls_df['store_longitude'].min()

print('Max value lat: ', max_value_lat)
print('Min value lat: ', min_value_lat)
print('Max value lng: ', max_value_lng)
print('Min value lng: ', min_value_lng)

Max value lat:  52.48624299999999
Min value lat:  32.4595933
Max value lng:  -1.890401
Min value lng:  -117.3754942


In [ ]:
# finding incorrect stores
outside_df = ls_df[(ls_df['store_latitude'] < 40.3622) | (ls_df['store_latitude'] > 43.5008) |
                   (ls_df['store_longitude'] < -96.6357) | (ls_df['store_longitude'] > -90.1538)]
outside_df_unique = outside_df.drop_duplicates(subset='unique_store')
outside_df_unique

,date,store_name,address,city,zip_code,county,category_name,vendor_name,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),unique_store,store_latitude,store_longitude
1486,07/05/2022,CASEY'S GENERAL STORE #2493 / BUFFALO,222 W FRONT ST,BUFFALO,52728,SCOTT,MIXTO TEQUILA,PROXIMO,JOSE CUERVO ESPECIAL SILVER,12,750,11.50,17.25,1,17.25,0.75,"222 W FRONT ST, BUFFALO, 52728",42.886447,-78.878369
9090,07/06/2022,CORK 'N BOTTLE / MANCHESTER,900 E MAIN ST,MANCHESTER,52087,DELAWARE,IMPORTED FLAVORED VODKA,CONSTELLATION BRANDS INC,SVEDKA STRAWBERRY LEMONADE,12,750,8.00,12.00,1,12.00,0.75,"900 E MAIN ST, MANCHESTER, 52087",41.775930,-72.521501
9508,07/06/2022,FAREWAY STORES #792 / TOLEDO,1005 S COUNTY LINE RD,TOLEDO,52342,TAMA,CANADIAN WHISKIES,DIAGEO AMERICAS,CROWN ROYAL,12,750,17.49,26.24,6,157.44,4.50,"1005 S COUNTY LINE RD, TOLEDO, 52342",41.652805,-83.537867
10259,07/06/2022,PRONTO MARKET,201 S MAIN ST,HAZLETON,50641,BUCHANAN,BLENDED WHISKIES,DIAGEO AMERICAS,SEAGRAMS 7 CROWN PET FLASK,12,750,7.50,11.25,3,33.75,2.25,"201 S MAIN ST, HAZLETON, 50641",38.488936,-87.541689
30907,07/08/2022,HY-VEE FOOD STORE / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302,LINN,WHISKEY LIQUEUR,SAZERAC COMPANY INC,FIREBALL CINNAMON WHISKEY,48,200,2.50,3.75,12,45.00,2.40,"3600 BUSINESS HWY 151 EAST, MARION, 52302",41.127599,-72.340083
39973,07/09/2022,CASEY'S GENERAL STORE # 2698/ PERRY,1308 1ST STREET,PERRY,50220,DALLAS,FLAVORED RUM,DIAGEO AMERICAS,CAPTAIN MORGAN SLICED APPLE MINI,12,50,4.80,7.20,2,14.40,0.10,"1308 1ST STREET, PERRY, 50220",32.459593,-83.729542
41766,07/09/2022,CASEY'S GENERAL STORE #1028 / MIDDLETOWN,75 MAIN ST,MIDDLETOWN,52638,DES MOINES,TEMPORARY & SPECIALTY PACKAGES,DIAGEO AMERICAS,CROWN ROYAL PEACH,12,750,17.49,26.24,12,314.88,9.00,"75 MAIN ST, MIDDLETOWN, 52638",41.562321,-72.650649
48510,07/11/2022,CASEY'S GENERAL STORE #1446 / LISBON,150 E. BUSINESS 30,LISBON,52253,LINN,TENNESSEE WHISKIES,BROWN FORMAN CORP.,JACK DANIELS OLD #7 BLACK LABEL,12,750,15.91,23.87,5,119.35,3.75,"150 E. BUSINESS 30, LISBON, 52253",38.722252,-9.139337
49195,07/11/2022,SPARKY'S ONE STOP / LAKE VIEW,HWY 71 & HWY 175,LAKE VIEW,51450,SAC,SPICED RUM,DIAGEO AMERICAS,CAPTAIN MORGAN ORIGINAL SPICED PET,12,750,9.06,13.59,12,163.08,9.00,"HWY 71 & HWY 175, LAKE VIEW, 51450",34.901078,-94.107443
49487,07/11/2022,EICHMAN ENTERPRISES INC / SAGEVILLE,11941 HWY 52 N,SAGEVILLE,52002,DUBUQUE,COCKTAILS/RTD,PROXIMO,JOSE CUERVO AUTHENTIC LIME LIGHT,6,200,4.64,6.96,6,41.76,1.20,"11941 HWY 52 N, SAGEVILLE, 52002",39.433039,-85.023317


In [ ]:
# update coordinates manually

# dictionary mapping store names to new coordinates
new_coordinates = {
    "222 W FRONT ST, BUFFALO, 52728": (41.45630372834186, -90.71875207116399),
    "1005 S COUNTY LINE RD, TOLEDO, 52342": (41.984928350468785, -92.58052348465577),
    "201 S MAIN ST, HAZLETON, 50641": (42.61544161440063, -91.90513158849606),
    "3600 BUSINESS HWY 151 EAST, MARION, 523022": (42.038025652180124, -91.57205743862289),
    "1308 1ST STREET, PERRY, 50220": (41.841119627707165, -94.1063095),
    "75 MAIN ST, MIDDLETOWN, 52638": (40.826718309099775, -91.25477228465593),
    "150 E. BUSINESS 30, LISBON, 52253": (41.91919615849863, -91.38414830793371),
    "HWY 71 & HWY 175, LAKE VIEW, 51450": (40.57757658631722, -95.0306485443796),
    "1101, JEFFERSON ST, DENVER, 50622": (42.66194279255661, -92.33644172275255),
    "113 HWY 30 WEST, TOLEDO, 52342": (41.986215934076725, -92.58046829999999),
    "121 SOUTH HWY 1, BIRMINGHAM, 52535": (40.87903554575884, -91.94892185396779),
    "305 E HWY 34, DANVILLE, 52623": (40.86447123349892, -91.31127920000016),
    "200 E 1ST ST, RIVERSIDE, 52327": (41.48018045779956, -91.57796013862287),
}

# update the DataFrame
for store, (new_latitude, new_longitude) in new_coordinates.items():
    ls_df.loc[ls_df['unique_store'] == store, 'store_latitude'] = new_latitude
    ls_df.loc[ls_df['unique_store'] == store, 'store_longitude'] = new_longitude

In [ ]:
# check for last remaing stores that couldn't be updated manually
outside_df = ls_df[(ls_df['store_latitude'] < 40.3622) | (ls_df['store_latitude'] > 43.5008) |
                   (ls_df['store_longitude'] < -96.6357) | (ls_df['store_longitude'] > -90.1538)]
outside_df_unique = outside_df.drop_duplicates(subset='unique_store')
outside_df_unique

,date,store_name,address,city,zip_code,county,category_name,vendor_name,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),unique_store,store_latitude,store_longitude
9090,07/06/2022,CORK 'N BOTTLE / MANCHESTER,900 E MAIN ST,MANCHESTER,52087,DELAWARE,IMPORTED FLAVORED VODKA,CONSTELLATION BRANDS INC,SVEDKA STRAWBERRY LEMONADE,12,750,8.00,12.00,1,12.00,0.75,"900 E MAIN ST, MANCHESTER, 52087",41.775930,-72.521501
30907,07/08/2022,HY-VEE FOOD STORE / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302,LINN,WHISKEY LIQUEUR,SAZERAC COMPANY INC,FIREBALL CINNAMON WHISKEY,48,200,2.50,3.75,12,45.00,2.40,"3600 BUSINESS HWY 151 EAST, MARION, 52302",41.127599,-72.340083
49487,07/11/2022,EICHMAN ENTERPRISES INC / SAGEVILLE,11941 HWY 52 N,SAGEVILLE,52002,DUBUQUE,COCKTAILS/RTD,PROXIMO,JOSE CUERVO AUTHENTIC LIME LIGHT,6,200,4.64,6.96,6,41.76,1.20,"11941 HWY 52 N, SAGEVILLE, 52002",39.433039,-85.023317


In [ ]:
# stores to drop as unable to manually update
strings_to_remove = ['900 E MAIN ST, MANCHESTER, 52087', '3600 BUSINESS HWY 151 EAST, MARION, 52302', '11941 HWY 52 N, SAGEVILLE, 52002']

# This will remove rows where 'column_name' is in 'strings_to_remove'
ls_df = ls_df[~ls_df['unique_store'].isin(strings_to_remove)]

In [ ]:
ls_df.shape

(2634425, 19)

In [ ]:
# exporting merged dataset
ls_df.to_csv('/content/drive/MyDrive/Iowa Liquor /Prepared Data/clean_liquor_sales.csv')

### B. County Demographics

In [ ]:
# importing data
county_df =  pd.read_csv('/content/drive/MyDrive/Iowa Liquor /Raw Data/county_demo.csv')
county_df.head()

,county,county_population,county_poverty_rate,county_median_earnings_(dollars)
0,Adair,7494,11.5,36126
1,Adams,3611,11.7,36524
2,Allamakee,13960,10.5,38212
3,Appanoose,12094,19.4,33276
4,Audubon,5598,11.3,32647


In [ ]:
# cleaning columns for merge
county_df['county'] = county_df['county'].str.upper()

In [ ]:
# merging county_df and ls_df
df_merged = ls_df.merge(county_df, on = 'county')
df_merged.shape

(2634425, 22)

In [ ]:
df_merged.head()

,date,store_name,address,city,zip_code,county,category_name,vendor_name,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),unique_store,store_latitude,store_longitude,county_population,county_poverty_rate,county_median_earnings_(dollars)
0,07/04/2022,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021,POLK,IMPORTED VODKAS,DIAGEO AMERICAS,KETEL ONE,6,1750,20.99,31.49,6,188.94,10.5,"410 NORTH ANKENY BLVD, ANKENY, 50021",41.734504,-93.602470,501089,10.1,42262
1,07/04/2022,HY-VEE WINE AND SPIRITS / ANKENY,410 NORTH ANKENY BLVD,ANKENY,50021,POLK,AMERICAN FLAVORED VODKA,DIAGEO AMERICAS,"SMIRNOFF RED, WHITE & BERRY",6,1750,14.75,22.13,6,132.78,10.5,"410 NORTH ANKENY BLVD, ANKENY, 50021",41.734504,-93.602470,501089,10.1,42262
2,07/04/2022,HY-VEE WINE AND SPIRITS / HUBBELL,2310 HUBBELL AVE,DES MOINES,50317,POLK,COCKTAILS/RTD,PROXIMO,JOSE CUERVO AUTHENTIC LIME MARGARITA,6,1750,8.74,13.11,6,78.66,10.5,"2310 HUBBELL AVE, DES MOINES, 50317",41.604613,-93.574132,501089,10.1,42262
3,07/04/2022,UNIVERSITY LIQUORS / WDM,9250 UNIVERSITY AVE UNIT 115,WEST DES MOINES,50266,POLK,AMERICAN SCHNAPPS,JIM BEAM BRANDS,DEKUYPER HOT DAMN!,12,750,6.75,10.13,6,60.78,4.5,"9250 UNIVERSITY AVE UNIT 115, WEST DES MOINES,...",41.599689,-93.834562,501089,10.1,42262
4,07/04/2022,"CENTRAL CITY LIQUOR, INC.",1460 2ND AVE,DES MOINES,50314,POLK,IMPORTED VODKAS,JIM BEAM BRANDS,EFFEN 80PRF,6,750,12.50,18.75,6,112.50,4.5,"1460 2ND AVE, DES MOINES, 50314",41.605652,-93.620233,501089,10.1,42262


### C. City Population

In [ ]:
# importing data
city_df =  pd.read_csv('/content/drive/MyDrive/Iowa Liquor /Raw Data/city_population.csv')
city_df.head()

,city,city_population
0,Ackley,1564
1,Ackworth,118
2,Adair,794
3,Adel,6453
4,Afton,853


In [ ]:
# cleaning columns for merge
city_df['city'] = city_df['city'].str.upper()

In [ ]:
# merging df_merged and city_df
df_merged = df_merged.merge(city_df, how='left', on='city')
df_merged.shape

(2634425, 23)

In [ ]:
# checking for missing values
df_merged.isnull().sum()

date                                    0
store_name                              0
address                                 0
city                                    0
zip_code                                0
county                                  0
category_name                           0
vendor_name                             0
item_description                        0
pack                                    0
bottle_volume_(ml)                      0
state_bottle_cost                       0
state_bottle_retail                     0
bottles_sold                            0
sale_(dollars)                          0
volume_sold_(liters)                    0
unique_store                            0
store_latitude                          0
store_longitude                         0
county_population                       0
county_poverty_rate                     0
county_median_earnings_(dollars)        0
city_population                     18071
dtype: int64

In [ ]:
# checking which cities are not matching up for the merge
nan_df = df_merged[df_merged.isna().any(axis=1)]
unique_values = nan_df['city'].unique()
unique_values

array(['LE CLAIRE', 'PLEASANT VALLEY', 'TROY MILLS', 'SAINT ANSGAR',
       'ST ANSGAR', 'GRAND MOUNDS', 'ST CHARLES', 'ST LUCAS', 'CLEARLAKE',
       'MT PLEASANT', 'JEWELL'], dtype=object)

In [ ]:
# pleasant valley is actually a suburb of davenport, renaming other cities for merge
df_merged['city'] = df_merged['city'].replace({'PLEASANT VALLEY': 'DAVENPORT',
                                               'SAINT ANSGAR': 'ST ANSGAR',
                                               'GRAND MOUNDS': 'GRAND MOUND',
                                               'CLEARLAKE': 'CLEAR LAKE',
                                               'JEWELL': 'JEWELL JUNCTION',
                                               'LE CLAIRE': 'LECLAIRE',
                                               'MOUNT PLEASANT': 'MT PLEASANT'})
city_df['city'] = city_df['city'].replace({'LE CLAIRE': 'LECLAIRE',
                                           'ST. ANSGAR': 'ST ANSGAR',
                                           'ST. CHARLES': 'ST CHARLES',
                                           'ST. LUCAS': 'ST LUCAS',
                                           'MOUNT PLEASANT': 'MT PLEASANT'})
# adding values for troy mills
new_data = {'city': ['TROY MILLS'], 'city_population': [300]}
new_df = pd.DataFrame(new_data)
city_df = city_df.append(new_df, ignore_index=True)

In [ ]:
# retrying merge
df_merged = df_merged.merge(city_df, how='left', on='city')
df_merged.shape

(2634425, 24)

In [ ]:
df_merged.isnull().sum()

date                                    0
store_name                              0
address                                 0
city                                    0
zip_code                                0
county                                  0
category_name                           0
vendor_name                             0
item_description                        0
pack                                    0
bottle_volume_(ml)                      0
state_bottle_cost                       0
state_bottle_retail                     0
bottles_sold                            0
sale_(dollars)                          0
volume_sold_(liters)                    0
unique_store                            0
store_latitude                          0
store_longitude                         0
county_population                       0
county_poverty_rate                     0
county_median_earnings_(dollars)        0
city_population_x                   18071
city_population_y                 

In [ ]:
# dropping and renaming columns
df_merged = df_merged.drop('city_population_x', axis=1)
df_merged = df_merged.rename(columns={'city_population_y': 'city_population'})

In [ ]:
# exporting data
df_merged.to_csv('/content/drive/MyDrive/Iowa Liquor /Prepared Data/clean_liquor_demographics.csv')

### D. Colleges

In [ ]:
# importing data
college_df =  pd.read_csv('/content/drive/MyDrive/Iowa Liquor /Raw Data/colleges.csv')
college_df.head()

,college_name,city,colleges_in_city,students,students_in_city
0,Allen College,Waterloo,2,678,5720
1,Briar Cliff University,Sioux City,4,1076,9058
2,Buena Vista University,Storm Lake,1,1863,1863
3,Central College,Pella,1,1120,1120
4,Clarke University,Dubuque,5,855,4836


In [ ]:
# adding coordinates to colleges using google maps
def get_coordinates(row):
    try:
        geocode_result = gmaps.geocode(f"{row['college_name']}, {row['city']}")
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return pd.Series((location['lat'], location['lng']))
        else:
            return pd.Series((np.nan, np.nan))
    except Exception as e:
        print(f"Error: {e}")
        return pd.Series((np.nan, np.nan))

# apply the function to the DataFrame
college_df[['college_latitude', 'college_longitude']] = college_df.apply(get_coordinates, axis=1)
college_df.head()

,college_name,city,colleges_in_city,students,students_in_city,college_latitude,college_longitude
0,Allen College,Waterloo,2,678,5720,42.530276,-92.340097
1,Briar Cliff University,Sioux City,4,1076,9058,42.527010,-96.427039
2,Buena Vista University,Storm Lake,1,1863,1863,42.641043,-95.209735
3,Central College,Pella,1,1120,1120,41.403022,-92.925837
4,Clarke University,Dubuque,5,855,4836,42.509414,-90.691447


In [ ]:
# cleaning columns for merge
college_df['city'] = college_df['city'].str.upper()

In [ ]:
# merging merged_df and college_df
college_agg = college_df.groupby('city').agg({'students_in_city': 'mean', 'colleges_in_city': 'mean'}).reset_index()
df_merged = df_merged.merge(college_agg, on='city', how='left')

In [ ]:
df_merged.shape

(2634425, 25)

In [ ]:
# replacing NaNs with 0 in 'students_in_city' and 'colleges_in_city' columns
df_merged['students_in_city'] = df_merged['students_in_city'].fillna(0)
df_merged['colleges_in_city'] = df_merged['colleges_in_city'].fillna(0)

In [ ]:
# exporting dataset
df_merged.to_csv('/content/drive/MyDrive/Iowa Liquor /Prepared Data/liquor_demo_college_clean.csv')

### E. Adding Nearest College Information

In [ ]:
df_merged = pd.read_csv('/content/drive/MyDrive/Iowa Liquor /Prepared Data/liquor_demo_college_clean.csv', index_col=0)

In [ ]:
# install haversine for distance calculation
!pip install haversine
from haversine import haversine, Unit

In [ ]:
# create a DataFrame with unique stores and their coordinates
unique_stores_df = df_merged[['unique_store', 'store_latitude', 'store_longitude']].drop_duplicates()

# function to calculate the distance to the nearest college and return its name
def closest_college(row, college_df):
    store_coords = (row['store_latitude'], row['store_longitude'])
    distances = college_df.apply(lambda x: haversine(store_coords, (x['college_latitude'], x['college_longitude']), unit=Unit.KILOMETERS), axis=1)
    min_distance = distances.min()
    nearest_college = college_df.loc[distances.idxmin(), 'college_name']
    nearest_college_students = college_df.loc[distances.idxmin(), 'students']
    return min_distance, nearest_college, nearest_college_students

# calculate the distance to the nearest college for each unique store and get the name of the college
unique_stores_df[['closest_college_kilometers', 'nearest_college', 'students_nearest_college']] = unique_stores_df.apply(lambda x: closest_college(x, college_df), axis=1, result_type='expand')

# merge these results back to the original DataFrame
df_merged = df_merged.merge(unique_stores_df[['unique_store', 'closest_college_kilometers', 'nearest_college', 'students_nearest_college']], on='unique_store', how='left')

In [ ]:
df_merged.shape

(2634425, 25)

### F. Adding Competitor Information

In [ ]:
# create a DataFrame with unique stores and their coordinates
unique_stores_df = df_merged[['unique_store', 'store_latitude', 'store_longitude']].drop_duplicates()

# function to calculate number of stores within 10 kilometers
def competitor_count(store_coords, store_df):
    distances = store_df.apply(lambda x: haversine(store_coords, (x['store_latitude'], x['store_longitude']), unit=Unit.KILOMETERS), axis=1)
    return (distances < 10).sum() - 1  # subtract 1 to exclude the store itself

# apply the function to each unique store
unique_stores_df['competitors_within_10_kilometers'] = unique_stores_df.apply(lambda x: competitor_count((x['store_latitude'], x['store_longitude']), unique_stores_df), axis=1)

# map these competitor counts back to the original DataFrame
df_merged = df_merged.merge(unique_stores_df[['unique_store', 'competitors_within_10_kilometers']], on='unique_store', how='left')


In [ ]:
df_merged.shape

(2634425, 29)

In [ ]:
# create a DataFrame with unique stores and their coordinates
unique_stores_df = df_merged[['unique_store', 'store_latitude', 'store_longitude']].drop_duplicates()

# function to calculate number of stores within 5 kilometers
def competitor_count(store_coords, store_df):
    distances = store_df.apply(lambda x: haversine(store_coords, (x['store_latitude'], x['store_longitude']), unit=Unit.KILOMETERS), axis=1)
    return (distances < 5).sum() - 1  # subtract 1 to exclude the store itself

# apply the function to each unique store
unique_stores_df['competitors_within_5_kilometers'] = unique_stores_df.apply(lambda x: competitor_count((x['store_latitude'], x['store_longitude']), unique_stores_df), axis=1)

# map these competitor counts back to the original DataFrame
df_merged = df_merged.merge(unique_stores_df[['unique_store', 'competitors_within_5_kilometers']], on='unique_store', how='left')


In [ ]:
# create a DataFrame with unique stores and their coordinates
unique_stores_df = df_merged[['unique_store', 'store_latitude', 'store_longitude']].drop_duplicates()

# function to calculate number of stores within 2 kilometers
def competitor_count(store_coords, store_df):
    distances = store_df.apply(lambda x: haversine(store_coords, (x['store_latitude'], x['store_longitude']), unit=Unit.KILOMETERS), axis=1)
    return (distances < 2).sum() - 1  # subtract 1 to exclude the store itself

# apply the function to each unique store
unique_stores_df['competitors_within_2_kilometers'] = unique_stores_df.apply(lambda x: competitor_count((x['store_latitude'], x['store_longitude']), unique_stores_df), axis=1)

# map these competitor counts back to the original DataFrame
df_merged = df_merged.merge(unique_stores_df[['unique_store', 'competitors_within_2_kilometers']], on='unique_store', how='left')

In [ ]:
df_merged.shape

(2634425, 31)

In [ ]:
# exporting dataset
df_merged.to_csv('/content/drive/MyDrive/Iowa Liquor /Prepared Data/liquor_demo_college_dist_clean.csv')